# 크롤링작업

In [ ]:
%pip install pandas requests lxml tqdm

## 중앙부처 - 크롤링

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import ssl
import time
import os
from requests.adapters import HTTPAdapter

# ==========================
# 사용자 입력
# ==========================
API_KEY = "0f561bd1f21a64f960fff4cf634d3afeb996d252a52dae3c7fea3c70c2c9ba09"   # ← 반드시 본인 API Key로 교체
MASTER_CSV = "중앙부처_복지서비스.csv"     # 전체 서비스ID 목록
PROCESSED_CSV = "중앙부처_batch4.csv"    # 이미 처리된 결과 CSV
SERVICE_ID_COLUMN = "서비스ID"
BASE_URL = "https://apis.data.go.kr/B554287/NationalWelfareInformationsV001/NationalWelfaredetailedV001"

# ==========================
# TLS 강제
# ==========================
class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        ctx.minimum_version = ssl.TLSVersion.TLSv1_2
        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

session = requests.Session()
session.mount("https://", TLSAdapter())

# ==========================
# XML 파싱
# ==========================
def parse_service_detail(xml_text):
    try:
        root = ET.fromstring(xml_text)
        node = root if root.tag == "wantedDtl" else root.find(".//wantedDtl")
        if node is None:
            return None
        row = {
            "서비스ID": node.findtext("servId", ""),
            "서비스명": node.findtext("servNm", ""),
            "소관부처": node.findtext("jurMnofNm", ""),
            "서비스개요": node.findtext("wlfareInfoOutlCn", ""),
            "지원대상상세": node.findtext("tgtrDtlCn", "").strip(),
            "선정기준": node.findtext("slctCritCn", "").strip(),
            "지원내용": node.findtext("alwServCn", "").strip(),
            "지원주기": node.findtext("sprtCycNm", ""),
            "지급방식": node.findtext("srvPvsnNm", ""),
        }
        return row
    except:
        return None

# ==========================
# API 호출
# ==========================
def fetch_service_detail(sid):
    params = {"serviceKey": API_KEY, "servId": sid}
    try:
        resp = session.get(BASE_URL, params=params, timeout=30)
        if resp.status_code == 200:
            return resp.text
        else:
            print(f"❌ 요청 실패: {sid}, 코드 {resp.status_code}")
            return None
    except Exception as e:
        print(f"⚠️ 요청 오류: {sid}, {e}")
        return None

# ==========================
# 실행 루프
# ==========================
def run_batch(service_ids, batch_num=1, batch_size=100):
    rows, fail_ids = [], []
    last_id = None

    for idx, sid in enumerate(service_ids, start=1):
        print(f"[{idx}/{len(service_ids)}] 요청: {sid}")
        xml_text = fetch_service_detail(sid)

        if not xml_text:
            fail_ids.append(sid)
            continue

        data = parse_service_detail(xml_text)
        if data:
            rows.append(data)
            last_id = sid
        else:
            fail_ids.append(sid)

        time.sleep(0.5)  # API 부하 방지

        # 100개 단위 저장
        if idx % batch_size == 0:
            file_name = f"중앙부처_batch{batch_num}.csv"
            pd.DataFrame(rows).to_csv(file_name, index=False, encoding="utf-8-sig")
            print(f"💾 Batch {batch_num} 저장 완료 ({len(rows)}개)")
            rows = []
            batch_num += 1

    # 남은 데이터 저장
    if rows:
        file_name = f"중앙부처_batch{batch_num}.csv"
        pd.DataFrame(rows).to_csv(file_name, index=False, encoding="utf-8-sig")
        print(f"💾 Batch {batch_num} 저장 완료 ({len(rows)}개)")

    # 실패 ID 저장
    if fail_ids:
        fail_file = f"중앙부처_batch{batch_num}_실패.csv"
        pd.DataFrame(fail_ids, columns=["실패ID"]).to_csv(fail_file, index=False, encoding="utf-8-sig")
        print(f"⚠️ 실패한 ID {len(fail_ids)}건 저장됨 → {fail_file}")

    print(f"✅ 마지막 처리된 서비스ID: {last_id}")


# ==========================
# 시작 index 계산
# ==========================
def get_remaining_ids(master_csv, processed_csv):
    df_master = pd.read_csv(master_csv)
    df_processed = pd.read_csv(processed_csv)

    all_ids = df_master[SERVICE_ID_COLUMN].dropna().astype(str).tolist()
    done_ids = df_processed[SERVICE_ID_COLUMN].dropna().astype(str).tolist()

    remaining = [sid for sid in all_ids if sid not in done_ids]
    print(f"📊 전체 {len(all_ids)}개 중 {len(done_ids)}개 완료, {len(remaining)}개 남음")
    return remaining


# ==========================
# 실행
# ==========================
if __name__ == "__main__":
    remaining_ids = get_remaining_ids(MASTER_CSV, PROCESSED_CSV)

    if remaining_ids:
        run_batch(remaining_ids, batch_num=3, batch_size=100)  
        # 👉 batch_num=3부터 시작 (1,2는 이미 완료했으니까)
    else:
        print("🎉 모든 서비스ID 크롤링 완료됨")


중앙부처 실패 id 합본

In [ ]:
import pandas as pd
import glob
import os

# --- 설정 ---

# 1. 찾을 실패 파일들의 패턴입니다. 
#    이 패턴에 맞는 모든 파일을 찾아냅니다. (예: 중앙부처_batch4_실패.csv, 중앙부처_재시도_재실패.csv 등)
FILE_PATTERN = "*_실패.csv"

# 2. 최종적으로 저장될 통합 파일의 이름입니다.
OUTPUT_FILENAME = "통합_실패_ID_목록.csv"

# 3. 통합 후 사용할 최종 서비스 ID 컬럼 이름입니다.
FINAL_ID_COLUMN_NAME = "서비스ID"

# --- 코드 실행 ---

def merge_failure_files():
    """
    지정된 패턴과 일치하는 모든 실패 CSV 파일을 찾아,
    하나의 통합된 파일로 병합하고 중복을 제거합니다.
    """
    # 패턴에 맞는 파일 목록을 찾습니다.
    failure_files = glob.glob(FILE_PATTERN)

    if not failure_files:
        print(f"❌ 오류: '{FILE_PATTERN}' 패턴에 맞는 실패 파일을 찾을 수 없습니다.")
        print("스크립트와 실패 파일들이 같은 폴더에 있는지 확인해주세요.")
        return

    print(f"📁 총 {len(failure_files)}개의 실패 파일을 찾았습니다:")
    for f in failure_files:
        print(f"  - {f}")

    df_list = []
    # 각 파일을 읽어 데이터프레임 리스트에 추가합니다.
    for file in failure_files:
        try:
            df = pd.read_csv(file)
            # 컬럼 이름이 통일되도록 처리
            # 예: '실패ID', '재실패ID' -> '서비스ID'
            if '실패ID' in df.columns:
                df.rename(columns={'실패ID': FINAL_ID_COLUMN_NAME}, inplace=True)
            elif '재실패ID' in df.columns:
                df.rename(columns={'재실패ID': FINAL_ID_COLUMN_NAME}, inplace=True)
            
            df_list.append(df)
        except Exception as e:
            print(f"⚠️ 경고: '{file}' 파일을 읽는 중 오류 발생. 건너뜁니다. ({e})")

    if not df_list:
        print("\n❌ 오류: 파일을 읽어오지 못했습니다.")
        return

    # 모든 데이터프레임을 하나로 합칩니다.
    print("\n파일 병합 및 중복 제거를 시작합니다...")
    combined_df = pd.concat(df_list, ignore_index=True)

    # 최종 ID 컬럼이 있는지 확인하고 중복을 제거합니다.
    if FINAL_ID_COLUMN_NAME in combined_df.columns:
        initial_rows = len(combined_df)
        # 중복된 ID를 제거하고, 최종적으로 유니크한 ID 목록만 남깁니다.
        combined_df.drop_duplicates(subset=[FINAL_ID_COLUMN_NAME], keep='first', inplace=True)
        final_rows = len(combined_df)
        print(f"  - 총 {initial_rows}개의 ID를 {final_rows}개의 고유 ID로 정리했습니다.")
    else:
        print(f"⚠️ 경고: '{FINAL_ID_COLUMN_NAME}' 컬럼을 찾을 수 없어 중복 제거를 건너뜁니다.")
        
    # 최종 결과를 CSV 파일로 저장합니다.
    try:
        combined_df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8-sig')
        print(f"\n✅ 성공! '{OUTPUT_FILENAME}' 파일로 모든 실패 ID를 통합 저장했습니다.")
    except Exception as e:
        print(f"\n❌ 오류: 최종 파일을 저장하는 중 문제가 발생했습니다: {e}")


# 메인 함수 실행
if __name__ == "__main__":
    merge_failure_files()

중앙부처 실패id 재크롤링

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import ssl
import time
from requests.adapters import HTTPAdapter
import os
from tqdm import tqdm

# --- 💡 1. 여기에 모든 API 키를 입력하세요 ---
API_KEYS = [
    "0f561bd1f21a64f960fff4cf634d3afeb996d252a52dae3c7fea3c70c2c9ba09",
    "6c00ed313ec5456c58b8d55a6f5d12a65cd4984956e87be30c7a3ea22b8ca086"
    # 가지고 있는 모든 키를 추가할 수 있습니다.
]

# --- 💡 2. 입력 파일과 결과 파일 이름 설정 ---
INPUT_FILE = "통합_실패_ID_목록.csv"
ID_COLUMN = "서비스ID" # 통합 실패 파일의 ID 컬럼명

OUTPUT_SUCCESS_FILE = "최종_재시도_성공.csv"
OUTPUT_FAILURE_FILE = "최종_재시도_실패.csv"

# --- API 호출 설정 (수정 필요 없음) ---
BASE_URL = "https://www.bokjiro.go.kr/ssis-tbu/TWAT52005M/twataa/wlfareInfo/selectWlfareInfo.do"

# (이 아래 TLSAdapter, session 설정은 기존과 동일합니다)
class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        ctx.minimum_version = ssl.TLSVersion.TLSv1_2
        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

session = requests.Session()
session.mount("https://", TLSAdapter())

# (XML 파싱 함수는 기존과 동일합니다)
def parse_service_detail(xml_text):
    try:
        root = ET.fromstring(xml_text)
        node = root if root.tag == "wantedDtl" else root.find(".//wantedDtl")
        if node is None:
            return None
        return {
            "서비스ID": node.findtext("servId", ""), "서비스명": node.findtext("servNm", ""),
            "소관부처": node.findtext("jurMnofNm", ""), "서비스개요": node.findtext("wlfareInfoOutlCn", ""),
            "지원대상상세": node.findtext("tgtrDtlCn", "").strip(), "선정기준": node.findtext("slctCritCn", "").strip(),
            "지원내용": node.findtext("alwServCn", "").strip(), "지원주기": node.findtext("sprtCycNm", ""),
            "지급방식": node.findtext("srvPvsnNm", ""),
        }
    except Exception:
        return None

def fetch_service_detail(sid, api_key):
    """지정된 API 키로 서비스 상세 정보를 요청합니다."""
    params = {"serviceKey": api_key, "servId": sid}
    try:
        resp = session.get(BASE_URL, params=params, timeout=30)
        # API 한도 초과 에러는 보통 특정 문자열을 포함한 응답을 줍니다.
        # 실제 응답에 따라 'LIMIT_EXCEEDED' 또는 다른 문자열로 변경해야 할 수 있습니다.
        if "SERVICE KEY IS NOT REGISTERED" in resp.text or "LIMITED" in resp.text:
             return "LIMIT_EXCEEDED"
        return resp.text if resp.status_code == 200 else None
    except requests.exceptions.RequestException:
        return None

# --- 최종 실행 로직 ---
def run_final_retry():
    # 1. 재시도할 ID 목록 불러오기
    try:
        df = pd.read_csv(INPUT_FILE)
        service_ids = df[ID_COLUMN].dropna().astype(str).tolist()
    except FileNotFoundError:
        print(f"❌ 입력 파일 '{INPUT_FILE}'을 찾을 수 없습니다. 스크립트와 같은 폴더에 있는지 확인하세요.")
        return

    # 2. 실행 시작
    print(f"✅ 총 {len(service_ids)}개의 실패 ID에 대한 최종 재시도를 시작합니다.")
    print(f"🔑 사용 가능한 API 키: {len(API_KEYS)}개")

    successful_rows = []
    failed_ids = []
    
    current_key_index = 0
    pbar = tqdm(total=len(service_ids), desc="ID 처리 중")
    
    i = 0
    while i < len(service_ids):
        sid = service_ids[i]
        
        if current_key_index >= len(API_KEYS):
            print("\n⚠️ 모든 API 키의 사용 한도에 도달했거나 유효하지 않습니다. 작업을 중단합니다.")
            failed_ids.extend(service_ids[i:]) # 남은 ID는 모두 실패로 처리
            pbar.update(len(service_ids) - i)
            break

        api_key = API_KEYS[current_key_index]
        xml_text = fetch_service_detail(sid, api_key)

        if xml_text == "LIMIT_EXCEEDED":
            print(f"\n🔑 API 키 {current_key_index + 1}의 한도에 도달. 다음 키로 전환합니다.")
            current_key_index += 1
            continue  # 현재 ID(sid)를 다음 키로 다시 시도하기 위해 i를 증가시키지 않음

        elif xml_text is None:
            failed_ids.append(sid)
            pbar.set_postfix_str(f"{sid} 요청 실패")
            i += 1
            pbar.update(1)

        else:
            data = parse_service_detail(xml_text)
            if data:
                successful_rows.append(data)
            else:
                failed_ids.append(sid)
            pbar.set_postfix_str(f"{sid} 처리 완료")
            i += 1
            pbar.update(1)
            
        time.sleep(0.3) # 서버 부하 감소

    pbar.close()

    # 3. 결과 저장
    if successful_rows:
        pd.DataFrame(successful_rows).to_csv(OUTPUT_SUCCESS_FILE, index=False, encoding="utf-8-sig")
        print(f"\n💾 재시도 성공: {len(successful_rows)}건을 '{OUTPUT_SUCCESS_FILE}' 파일로 저장했습니다.")
    
    if failed_ids:
        pd.DataFrame(failed_ids, columns=["재실패ID"]).to_csv(OUTPUT_FAILURE_FILE, index=False)
        print(f"💾 재시도 실패: {len(failed_ids)}건을 '{OUTPUT_FAILURE_FILE}' 파일로 저장했습니다.")
        
    print("\n🎉 모든 작업이 완료되었습니다.")


if __name__ == "__main__":
    run_final_retry()

중앙부처 csv파일 합치기

In [ ]:
import pandas as pd

files = ['중앙부처_1.csv', '중앙부처_2.csv']

dfs = [pd.read_csv(f) for f in files]
merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv("중앙부처_1,2합본.csv", index=False, encoding='utf-8-sig')

print("✅ 통합 완료 → 중앙부처_1,2합본.csv")
print(f"총 행 개수: {len(merged_df)}")
print(f"컬럼: {list(merged_df.columns)}")

중앙부처 합본 -2차

In [ ]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm

# --- 설정 ---

# 1. 기존에 작업했던 합본 파일 이름
MAIN_CSV_FILE = "중앙부처_1,2합본.csv"

# 2. 폴더에서 찾을 XML 파일 패턴
XML_PATTERN = "*.xml"

# 3. 최종적으로 모든 데이터가 합쳐져 저장될 파일 이름
FINAL_OUTPUT_FILE = "최종_중앙부처_통합본.csv"

# --- XML 파싱 함수 (기존 코드와 동일) ---
def parse_service_detail(xml_text):
    """XML 텍스트를 입력받아 파싱하고, 데이터 딕셔너리를 반환합니다."""
    try:
        root = ET.fromstring(xml_text)
        node = root if root.tag == "wantedDtl" else root.find(".//wantedDtl")
        if node is None:
            return None
        return {
            "서비스ID": node.findtext("servId", ""),
            "서비스명": node.findtext("servNm", ""),
            "소관부처": node.findtext("jurMnofNm", ""),
            "서비스개요": node.findtext("wlfareInfoOutlCn", ""),
            "지원대상상세": node.findtext("tgtrDtlCn", "").strip(),
            "선정기준": node.findtext("slctCritCn", "").strip(),
            "지원내용": node.findtext("alwServCn", "").strip(),
            "지원주기": node.findtext("sprtCycNm", ""),
            "지급방식": node.findtext("srvPvsnNm", ""),
        }
    except ET.ParseError:
        # XML 형식이 잘못된 경우
        return None

# --- 메인 실행 로직 ---
def process_and_merge_files():
    # 1. XML 파일 목록 찾기
    xml_files = glob.glob(XML_PATTERN)
    if not xml_files:
        print(f"❌ 오류: 폴더에서 '{XML_PATTERN}' 패턴에 맞는 XML 파일을 찾을 수 없습니다.")
        return

    print(f"📁 총 {len(xml_files)}개의 XML 파일을 찾았습니다. 파싱을 시작합니다...")

    # 2. 각 XML 파일을 파싱하여 데이터 리스트 생성
    recovered_data = []
    for file_path in tqdm(xml_files, desc="XML 파일 처리 중"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                # 파일 내용이 비어있지 않은 경우에만 파싱 시도
                if content.strip():
                    data = parse_service_detail(content)
                    if data:
                        recovered_data.append(data)
        except Exception as e:
            print(f"⚠️ 경고: '{file_path}' 파일 처리 중 오류 발생. 건너뜁니다. ({e})")
            
    if not recovered_data:
        print("❌ 오류: XML 파일에서 유효한 데이터를 추출하지 못했습니다.")
        return
        
    recovered_df = pd.DataFrame(recovered_data)
    print(f"✅ {len(recovered_df)}개의 서비스 데이터를 XML 파일로부터 성공적으로 복구했습니다.")

    # 3. 기존 합본 CSV 파일 불러오기
    try:
        print(f"\n기존 합본 파일 '{MAIN_CSV_FILE}'을 불러옵니다...")
        main_df = pd.read_csv(MAIN_CSV_FILE)
    except FileNotFoundError:
        print(f"❌ 오류: '{MAIN_CSV_FILE}' 파일을 찾을 수 없습니다. 파일이 스크립트와 같은 폴더에 있는지 확인하세요.")
        return

    # 4. 복구된 데이터와 기존 데이터 병합
    print("복구된 데이터와 기존 데이터를 병합합니다...")
    final_df = pd.concat([main_df, recovered_df], ignore_index=True)
    
    initial_rows = len(final_df)
    print(f"  - 병합 후 총 행 개수: {initial_rows}개")

    # 5. 서비스ID 기준으로 중복 데이터 제거 (복구된 데이터가 우선순위를 가짐)
    final_df.drop_duplicates(subset=['서비스ID'], keep='last', inplace=True)
    final_rows = len(final_df)
    print(f"  - 중복 제거 후 최종 행 개수: {final_rows}개 ({initial_rows - final_rows}개 중복 제거)")

    # 6. 최종 결과를 새 파일로 저장
    try:
        final_df.to_csv(FINAL_OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"\n🎉 성공! 최종 통합된 데이터를 '{FINAL_OUTPUT_FILE}' 파일로 저장했습니다.")
    except Exception as e:
        print(f"\n❌ 오류: 최종 파일을 저장하는 중 문제가 발생했습니다: {e}")

# --- 스크립트 실행 ---
if __name__ == "__main__":
    process_and_merge_files()

중앙부처 합본-3차

In [ ]:
import pandas as pd

files = ['중앙부처_1.csv', '중앙부처_2.csv']

dfs = [pd.read_csv(f) for f in files]
merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv("중앙부처_1,2합본.csv", index=False, encoding='utf-8-sig')

print("✅ 통합 완료 → 중앙부처_1,2합본.csv")
print(f"총 행 개수: {len(merged_df)}")
print(f"컬럼: {list(merged_df.columns)}")

## 지자체 파일 크롤링-복지로사이트

In [ ]:
import time
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 입력 CSV 파일
MASTER_CSV = "지자체_복지서비스리스트_통합.csv"
SERVICE_ID_COLUMN = "서비스ID"

# URL 포맷
URL_TEMPLATE = "https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId={}&wlfareInfoReldBztpCd=02"

def scrape_service(driver, service_id):
    """단일 서비스 ID 크롤링"""
    url = URL_TEMPLATE.format(service_id)
    details = {
        "서비스ID": service_id,
        "카테고리": "",
        "지원대상": "",
        "선정기준": "",
        "서비스내용": "",
        "신청방법": ""
    }

    try:
        driver.get(url)

        # 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.cl-text"))
        )
        time.sleep(0.5)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 카테고리 추출
        categories = []
        category_divs = soup.find_all("div", class_="cl-text", style=lambda x: x and "vertical-align:inherit" in x)
        for div in category_divs:
            txt = div.get_text(strip=True)
            if txt not in ["지원대상", "선정기준", "서비스 내용", "신청방법"]:  # 본문 키 제외
                categories.append(txt)
        details["카테고리"] = ", ".join(categories)

        # 본문 추출
        mapping = {
            "지원대상": "지원대상",
            "선정기준": "선정기준",
            "서비스 내용": "서비스내용",
            "신청방법": "신청방법"
        }
        for title_div in soup.find_all("div", class_="cl-text"):
            title = title_div.get_text(strip=True)
            if title in mapping:
                content_div = title_div.find_next("div", style=lambda x: x and "overflow:hidden" in x)
                if content_div:
                    details[mapping[title]] = content_div.get_text(" ", strip=True)

    except Exception as e:
        print(f"❌ 오류 (ID: {service_id}): {e}")

    return details


def get_resume_point(batch_prefix="지자체_복지서비스_크롤링결과_batch"):
    """마지막 저장된 배치 파일을 찾아 이어하기 시작점 반환"""
    batch_files = [f for f in os.listdir() if f.startswith(batch_prefix) and f.endswith(".csv")]
    if not batch_files:
        return 0, 1  # (시작 index, 다음 batch 번호)

    batch_files.sort()
    last_file = batch_files[-1]
    print(f"📂 마지막 저장된 파일: {last_file}")

    try:
        df_last = pd.read_csv(last_file)
        if not df_last.empty:
            last_id = df_last["서비스ID"].iloc[-1]
            print(f"👉 마지막 저장된 서비스ID: {last_id}")
            return df_last.index[-1] + 1 + (len(batch_files)-1)*100, len(batch_files)+1
    except Exception as e:
        print(f"⚠️ 이어하기 파일 분석 중 오류: {e}")

    return 0, 1


def main(batch_size=100):
    # 서비스ID 목록 불러오기
    df_master = pd.read_csv(MASTER_CSV)
    service_ids = df_master[SERVICE_ID_COLUMN].dropna().astype(str).tolist()

    # 이어하기 시작점 확인
    start_idx, batch_num = get_resume_point()
    if start_idx >= len(service_ids):
        print("🎉 모든 서비스ID에 대한 크롤링이 이미 완료되었습니다.")
        return

    # 크롬 드라이버 준비
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    results = []
    last_id = None

    for idx, sid in enumerate(service_ids[start_idx:], start=start_idx+1):
        print(f"[{idx}/{len(service_ids)}] 크롤링 중: {sid}")
        data = scrape_service(driver, sid)
        results.append(data)
        last_id = sid

        # 100개 단위 저장
        if idx % batch_size == 0:
            out_df = pd.DataFrame(results)
            out_df.to_csv(f"지자체_복지서비스_크롤링결과_batch{batch_num}.csv", index=False, encoding="utf-8-sig")
            print(f"💾 Batch {batch_num} 저장 완료 ({len(results)}개)")
            batch_num += 1
            results = []

        time.sleep(0.5)  # 서버 부하 방지

    # 마지막 남은 결과 저장
    if results:
        out_df = pd.DataFrame(results)
        out_df.to_csv(f"지자체_복지서비스_크롤링결과_batch{batch_num}.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Batch {batch_num} 저장 완료 ({len(results)}개)")

    driver.quit()
    print(f"✅ 마지막 저장된 서비스ID: {last_id}")


if __name__ == "__main__":
    main(batch_size=100)

지자체 크롤링 통합

In [ ]:
import pandas as pd
import glob

# 합칠 파일 패턴 지정
file_list = sorted(glob.glob("지자체_복지서비스_크롤링결과_batch*.csv"))

print(f"총 {len(file_list)}개 파일을 병합합니다.")

# 파일 읽어서 합치기
dfs = [pd.read_csv(f) for f in file_list]
merged_df = pd.concat(dfs, ignore_index=True)

# 최종 저장
merged_df.to_csv("지자체_복지서비스_크롤링결과_통합본.csv", index=False, encoding="utf-8-sig")

print("✅ 병합 완료: 지자체_복지서비스_크롤링결과_통합본.csv")



지자체 csv 통합 - 세부내용 + 리스트

In [ ]:
import pandas as pd

# 파일 불러오기
base_df = pd.read_csv("지자체_복지서비스리스트_통합.csv", encoding="utf-8-sig")
crawl_df = pd.read_csv("지자체_복지서비스_크롤링결과_통합본.csv", encoding="utf-8-sig")

# 서비스ID 기준 병합 (좌측 기준)
merged_df = pd.merge(base_df, crawl_df, on="서비스ID", how="left")

# 저장 (최종 파일 이름 지정)
output_file = "local_final_welfare_list.csv"
merged_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ 병합 완료: {output_file}")
print(f"총 행 수: {len(merged_df)}")

## 민간복지리스트 크롤링

In [ ]:
import requests
import pandas as pd
import time

URL = "https://www.bokjiro.go.kr/ssis-tbu/TWAT52005M/twataa/wlfareInfo/selectWlfareInfo.do"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    "Referer": "https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52005M.do",
    "Content-Type": "application/json;charset=UTF-8",
}

def fetch_page(page: int):
    payload = {
        "dmSearchParam": {
            "page": str(page),
            "orderBy": "date",
            "tabId": "3",   # ✅ 민간 탭
            "searchTerm": "",
            "onlineYn": "",
            "bkjrLftmCycCd": "",
            "daesang": "",
            "period": "",
            "age": "",
            "region": "",
            "jjim": "",
            "subject": "",
            "favoriteKeyword": "Y",
            "sido": "",
            "gungu": "",
            "endYn": "N"
        },
        "dmScr": {
            "curScrId": "tbu/app/twat/twata/twataa/TWAT52005M",
            "befScrId": ""
        }
    }
    res = requests.post(URL, headers=headers, json=payload)
    res.raise_for_status()
    return res.json().get("dsServiceList3", [])

# ✅ 전체 페이지 크롤링
all_data = []
for page in range(1, 39):  # 1~38 페이지
    data = fetch_page(page)
    print(f"{page}페이지: {len(data)}건 수집")
    all_data.extend(data)
    time.sleep(0.3)

print("총 민간 데이터:", len(all_data))

# ✅ 기본 필드 + RETURN_STR 세부항목 파싱
rows = []
for svc in all_data:
    base_info = {
        "서비스ID": svc.get("WLFARE_INFO_ID"),
        "서비스명": svc.get("WLFARE_INFO_NM"),
        "서비스 요약": svc.get("WLFARE_INFO_OUTL_CN"),
        "상세 링크": f"https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId={svc.get('WLFARE_INFO_ID')}&wlfareInfoReldBztpCd=03",
        "소관기관": svc.get("BIZ_CHR_INST_NM"),
        "대표연락처": svc.get("RPRS_CTADR"),
        "신청가능여부": svc.get("ONLINEYN"),
        "주소": svc.get("ADDR"),
        "태그": svc.get("TAG_NM"),
        "시작일": svc.get("ENFC_BGNG_YMD"),
        "종료일": svc.get("ENFC_END_YMD"),
        "진행상태": svc.get("CVL_PROGRSS_STATUS"),
    }
    
    # RETURN_STR 파싱
    return_str = svc.get("RETURN_STR", "")
    parsed_info = {}
    if return_str:
        for item in return_str.split(";"):
            if ":" in item:
                k, v = item.split(":", 1)
                parsed_info[k.strip()] = v.strip()
    
    # 기본정보 + RETURN_STR 확장정보 병합
    rows.append({**base_info, **parsed_info})

# ✅ CSV 저장
df = pd.DataFrame(rows)
df.to_csv("민간_복지서비스_확장.csv", index=False, encoding="utf-8-sig")

print("📁 민간_복지서비스_확장.csv 저장 완료")

민간복지 상세내역 크롤링

In [ ]:
import time
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 입력 CSV 파일
MASTER_CSV = "민간_복지서비스_확장.csv"   # 민간 서비스 ID 들어있는 파일
SERVICE_ID_COLUMN = "서비스ID"

# URL 포맷
URL_TEMPLATE = "https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52015M.do?wlfareInfoId={}&wlfareInfoReldBztpCd=03"

def scrape_service(driver, service_id):
    """단일 민간 서비스 ID 상세 크롤링"""
    url = URL_TEMPLATE.format(service_id)
    details = {
        "서비스ID": service_id,
        "카테고리": "",
        "사업상태": "",
        "사업기간": "",
        "연락처": "",
        "이메일": "",
        "사업목적": "",
        "지원대상": "",
        "지원내용": "",
        "신청방법": "",
        "제출서류": ""
    }

    try:
        driver.get(url)

        # 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.cl-text"))
        )
        time.sleep(0.5)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ✅ 카테고리 (상단 badge-wlfare 영역)
        category_divs = soup.select("div.badge-wlfare div.cl-text")
        categories = [c.get_text(strip=True) for c in category_divs if c.get_text(strip=True)]
        details["카테고리"] = ", ".join(categories)

        # ✅ 단일 필드 (사업상태, 사업기간, 연락처, 이메일)
        mapping_simple = {
            "사업상태": "사업상태",
            "사업기간": "사업기간",
            "연락처": "연락처",
            "이메일": "이메일"
        }
        for title, col in mapping_simple.items():
            title_div = soup.find("div", class_="cl-text", string=title)
            if title_div:
                sibling_div = title_div.find_parent().find_next_sibling("div")
                if sibling_div and "cl-text" in sibling_div.get("class", []):
                    details[col] = sibling_div.get_text(" ", strip=True)

        # ✅ 본문 필드 (사업목적, 지원대상, 지원내용, 신청방법, 제출서류)
        mapping_long = {
            "사업목적": "사업목적",
            "지원대상": "지원대상",
            "지원내용": "지원내용",
            "신청방법": "신청방법",
            "제출서류": "제출서류"
        }
        for title, col in mapping_long.items():
            title_div = soup.find("div", class_="cl-text", string=title)
            if title_div:
                content_block = title_div.find_parent().find_next("div", class_="cl-control")
                if content_block:
                    text_div = content_block.find("div", class_="cl-text")
                    if text_div:
                        details[col] = text_div.get_text("\n", strip=True)

    except Exception as e:
        print(f"❌ 오류 (ID: {service_id}): {e}")

    return details


def get_resume_point(batch_prefix="민간_복지서비스_크롤링결과_batch"):
    """마지막 저장된 배치 파일을 찾아 이어하기 시작점 반환"""
    batch_files = [f for f in os.listdir() if f.startswith(batch_prefix) and f.endswith(".csv")]
    if not batch_files:
        return 0, 1  # (시작 index, 다음 batch 번호)

    batch_files.sort()
    last_file = batch_files[-1]
    print(f"📂 마지막 저장된 파일: {last_file}")

    try:
        df_last = pd.read_csv(last_file)
        if not df_last.empty:
            last_id = df_last["서비스ID"].iloc[-1]
            print(f"👉 마지막 저장된 서비스ID: {last_id}")
            return df_last.index[-1] + 1 + (len(batch_files)-1)*100, len(batch_files)+1
    except Exception as e:
        print(f"⚠️ 이어하기 파일 분석 중 오류: {e}")

    return 0, 1


def main(batch_size=100):
    # 서비스ID 목록 불러오기
    df_master = pd.read_csv(MASTER_CSV)
    service_ids = df_master[SERVICE_ID_COLUMN].dropna().astype(str).tolist()

    # 이어하기 시작점 확인
    start_idx, batch_num = get_resume_point()
    if start_idx >= len(service_ids):
        print("🎉 모든 서비스ID에 대한 크롤링이 이미 완료되었습니다.")
        return

    # 크롬 드라이버 준비
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    results = []
    last_id = None

    for idx, sid in enumerate(service_ids[start_idx:], start=start_idx+1):
        print(f"[{idx}/{len(service_ids)}] 크롤링 중: {sid}")
        data = scrape_service(driver, sid)
        results.append(data)
        last_id = sid

        # 100개 단위 저장
        if idx % batch_size == 0:
            out_df = pd.DataFrame(results)
            out_df.to_csv(f"민간_복지서비스_크롤링결과_batch{batch_num}.csv", index=False, encoding="utf-8-sig")
            print(f"💾 Batch {batch_num} 저장 완료 ({len(results)}개)")
            batch_num += 1
            results = []

        time.sleep(0.5)  # 서버 부하 방지

    # 마지막 남은 결과 저장
    if results:
        out_df = pd.DataFrame(results)
        out_df.to_csv(f"민간_복지서비스_크롤링결과_batch{batch_num}.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Batch {batch_num} 저장 완료 ({len(results)}개)")

    driver.quit()
    print(f"✅ 마지막 저장된 서비스ID: {last_id}")


if __name__ == "__main__":
    main(batch_size=100)

파일 합치기 - 상세내역만 나온것

In [ ]:
import pandas as pd
import glob

# 병합할 파일 패턴
file_pattern = "민간_복지서비스_크롤링결과_batch*.csv"

# 모든 배치 파일 경로 가져오기
files = sorted(glob.glob(file_pattern))

print("📂 합칠 파일:", files)

# 파일 읽어서 세로로 이어붙이기
dfs = [pd.read_csv(f) for f in files]
merged_df = pd.concat(dfs, ignore_index=True)

# 저장
output_file = "민간_복지서비스_크롤링결과_total.csv"
merged_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ 저장 완료: {output_file}, 총 {len(merged_df)} 행")

리스트 + 상세내역

In [ ]:
import pandas as pd

# 파일 경로
base_file = "민간_복지서비스_확장.csv"          # 기준 파일
crawl_file = "민간_복지서비스_크롤링결과_total.csv"  # 크롤링 결과 파일
output_file = "private_final_welfare_list.csv"

# CSV 읽기
df_base = pd.read_csv(base_file)
df_crawl = pd.read_csv(crawl_file)

# 기준 컬럼명 확인
print("기준 파일 컬럼:", df_base.columns.tolist())
print("크롤링 파일 컬럼:", df_crawl.columns.tolist())

# 서비스ID 기준으로 병합 (오른쪽 방향으로 붙이기)
df_merged = pd.merge(df_base, df_crawl, on="서비스ID", how="left")

# 저장
df_merged.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ 병합 완료! 저장된 파일: {output_file}")


# 전처리

## local_welfare 전처리정리

In [ ]:
import pandas as pd
import re

# CSV 불러오기
file_path = "local_final_welfare_list.csv"
df = pd.read_csv(file_path)

# ==========================
# 1. 카테고리 컬럼 불필요한 값 제거
# ==========================
remove_list = [
    "화면크기", "님", "로그아웃까지 남은 시간", "30:00",
    "대한민국 국민 누구나!", "지자체 복지서비스", "찜하기",
    "최종 수정일(반영일)", "지원주기", "제공유형", "처리절차"
]

def clean_category(val):
    if pd.isna(val):
        return val
    items = [x.strip() for x in str(val).split(",")]
    filtered = []
    for x in items:
        if len(x) == 10 and x[4] == "-" and x[7] == "-":
            continue
        if "회성" in x:
            continue
        if any(r == x or r in x for r in remove_list):
            continue
        filtered.append(x)
    return ", ".join(filtered)

df["카테고리"] = df["카테고리"].apply(clean_category)

# ==========================
# 2. 신청방법_x → 신청환경
# ==========================
if "신청방법_x" in df.columns:
    df.rename(columns={"신청방법_x": "신청환경"}, inplace=True)

    def normalize_env(val):
        if pd.isna(val):
            return val
        val = str(val).replace(" ", "")
        if "방문" in val and "인터넷" in val:
            return "ALL"
        elif "방문" in val:
            return "방문"
        elif "인터넷" in val:
            return "인터넷"
        return val

    df["신청환경"] = df["신청환경"].apply(normalize_env)

# ==========================
# 3. 신청방법_y → 신청방법상세
# ==========================
if "신청방법_y" in df.columns:
    df.rename(columns={"신청방법_y": "신청방법상세"}, inplace=True)

    def clean_detail(val):
        if pd.isna(val):
            return val
        text = str(val).strip()
        # 맨 앞 "-" 제거
        text = re.sub(r"^-\s*", "", text)
        # 나머지 "-" 를 "," 로 변환
        text = text.replace("-", ",")
        return text.strip()

    df= df.applymap(lambda x: clean_detail(x))

# ==========================
# 저장
# ==========================
output_path = "local_final_welfare_list_cleaned_copied.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 전처리 완료! 저장된 파일:", output_path)


칼럼 맞추기

In [ ]:
import pandas as pd

# CSV 불러오기
file_path = "local_final_welfare_list_cleaned_copied.csv"
df = pd.read_csv(file_path)

# 카테고리 사전 정의 (키워드 매핑)
category_keywords = {
    "생계지원": ["식비", "주거비", "공과금", "차상위", "생활비", "긴급복지", "할인"],
    "돌봄·보호": ["간병", "시설보호", "주야간", "보호자", "양육", "간호"],
    "건강·의료": ["질병", "진단", "치료", "의료비", "병원", "임신", "출산"],
    "일상생활": ["가사", "식사", "이동", "위생", "생활지원"],
    "안전위기": ["폭력", "학대", "자해", "위기", "긴급", "안전"],
    "주거지원": ["거처", "임대", "전세", "월세", "보증금", "주거환경"],
    "일자리": ["구직", "자활", "취업", "직업", "훈련", "능력개발"],
    "아동지원": ["아동", "보육", "교육", "양육상담", "돌봄"],
    "채무·법률": ["채무", "법률", "소송", "변호", "상담", "빚"],
    "고립·고독": ["고독", "고립", "사회적", "외로움", "고독사"]
}

# 카테고리 우선순위 정의
priority_order = [
    "생계지원", "주거지원", "일자리", "아동지원",
    "돌봄·보호", "건강·의료", "안전위기", "채무·법률", "고립·고독"
]

# 카테고리 분류 함수 (항상 하나 선택)
def assign_single_category(row):
    text = ""
    for col in ["서비스 요약", "서비스내용", "신청방법상세"]:
        if col in df.columns and pd.notna(row[col]):
            text += " " + str(row[col])
    matched = [cat for cat, keywords in category_keywords.items() if any(k in text for k in keywords)]
    for cat in priority_order:  # 우선순위대로 하나만 선택
        if cat in matched:
            return cat
    return "생계지원"  # 매칭 없으면 기본값

# 카테고리 갱신
df["카테고리"] = df.apply(assign_single_category, axis=1)

# 저장
output_path = "local_final_welfare_list_categorized.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 카테고리 분류 완료, 저장된 파일:", output_path)


## 중앙부처 전처리

In [ ]:
import pandas as pd
import re

# 파일 불러오기
file_path = "goverment_final_welfare_list_with_features.csv"
df = pd.read_csv(file_path)

# 1. 상세링크 추가
df["상세링크"] = df["서비스ID"].apply(
    lambda x: f"https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId={x}&wlfareInfoReldBztpCd=01"
)

# 2. 대상특성 추출 함수
def extract_target_feature(text):
    if not text:
        return ""
    text = str(text)

    # 나이 범위
    match_range = re.search(r"(\d{1,2})\s*~\s*(\d{1,2})세", text)
    if match_range:
        return f"{match_range.group(1)}~{match_range.group(2)}세"
    match_over = re.search(r"만?\s*(\d{1,2})세\s*이상", text)
    if match_over:
        return f"{match_over.group(1)}세 이상"
    match_under = re.search(r"만?\s*(\d{1,2})세\s*이하", text)
    if match_under:
        return f"{match_under.group(1)}세 이하"

    # 키워드 매핑
    if re.search(r"(청년|청소년)", text):
        return "청년"
    if re.search(r"(노인|노년|고령|어르신)", text):
        return "노년"
    if re.search(r"(저소득|차상위|기초생활|중위소득)", text):
        return "저소득"
    if re.search(r"(임신|출산|산모|임부)", text):
        return "임산부"

    return ""

# 지원대상상세 기반 대상특성 생성
if "지원대상상세" in df.columns:
    df["대상특성"] = df["지원대상상세"].apply(extract_target_feature)
else:
    df["대상특성"] = ""

# 3. 카테고리 분류
category_keywords = {
    "생계지원": ["식비", "주거비", "공과금", "생활비", "차상위", "긴급복지", "할인"],
    "돌봄·보호": ["간병", "시설보호", "주야간", "보호자", "양육", "간호"],
    "건강·의료": ["질병", "진단", "치료", "의료비", "병원", "임신", "출산"],
    "일상생활": ["가사", "식사", "이동", "위생", "생활지원"],
    "안전위기": ["폭력", "학대", "자해", "위기", "긴급", "안전"],
    "주거지원": ["거처", "임대", "전세", "월세", "보증금", "주거환경"],
    "일자리": ["구직", "자활", "취업", "직업", "훈련", "능력개발"],
    "아동지원": ["아동", "보육", "교육", "양육상담", "돌봄"],
    "채무·법률": ["채무", "법률", "소송", "변호", "상담", "빚"],
    "고립·고독": ["고독", "고립", "사회적", "외로움", "고독사"]
}

priority_order = [
    "생계지원", "주거지원", "일자리", "아동지원",
    "돌봄·보호", "건강·의료", "안전위기", "채무·법률", "고립·고독"
]

def assign_category(text):
    if not text:
        return "생계지원"  # 기본값
    text = str(text)
    matched = [cat for cat, keywords in category_keywords.items() if any(k in text for k in keywords)]
    for cat in priority_order:
        if cat in matched:
            return cat
    return "생계지원"

# 서비스개요와 서비스명 기준으로 카테고리 분류
if "서비스개요" in df.columns:
    df["카테고리"] = df["서비스개요"].apply(assign_category)
else:
    df["카테고리"] = df["서비스명"].apply(assign_category)

# 4. 저장
output_path = "goverment_final_welfare_list_categorized.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ CSV 저장 완료:", output_path)


민간복지 빈 칼럼 정리

In [ ]:
import pandas as pd
import os

# 원본 파일 경로
input_path = "private_final_welfare_list.csv"

# 출력 파일 경로
base, ext = os.path.splitext(input_path)
output_path = base + "_no_empty_cols" + ext

# 데이터 불러오기
df = pd.read_csv(input_path, dtype=object)  # 모든 칼럼을 object로 읽으면 공백처리 안전

def col_has_any_value(s: pd.Series) -> bool:
    """
    시리즈가 '모두 빈 값'인지 검사.
    - NaN은 빈값으로 처리
    - 문자열일 경우 공백만 있는 것도 빈값으로 처리
    - 숫자/기타 non-NaN 값은 값이 있는 것으로 간주
    """
    # NaN을 빈 문자열로 대체(벡터화)
    filled = s.where(s.notna(), "")
    # 문자열로 변환 후 양쪽 공백 제거
    stripped = filled.astype(str).str.strip()
    # 하나라도 빈문자열이 아닌 값이 있으면 True
    return (stripped != "").any()

# 각 컬럼에 대해 비어있는지 검사
has_value_mask = df.apply(col_has_any_value, axis=0)
kept_columns = has_value_mask[has_value_mask].index.tolist()
dropped_columns = has_value_mask[~has_value_mask].index.tolist()

print("유지할 컬럼 개수:", len(kept_columns))
print("제거할 빈 컬럼들 (총 {}개):".format(len(dropped_columns)))
for c in dropped_columns:
    print(" -", c)

# 빈 컬럼 제거한 데이터프레임
df_clean = df.loc[:, kept_columns]

# 저장
df_clean.to_csv(output_path, index=False, encoding="utf-8-sig")
print("저장 완료:", output_path)


민간복지 전처리-1

In [ ]:
import pandas as pd
import re
import os

input_path = "private_final_welfare_list.csv"
output_path = "private_final_welfare_list_standardized.csv"

# 안전하게 문자열로 읽기
df = pd.read_csv(input_path, dtype=str)

# (생략: 연령 추출 함수, 컬럼 rename 등 기존 코드 유지)
# --- 여기에 앞서 사용하신 derive_age_from_text, 나이 생성 코드 등을 넣으시면 됩니다 ---
# (예: derive_age_from_text 정의, '나이(코드기반)'/'나이' 생성 등)

# ----------------------------
# 빈 칼럼 검사 함수 (안전한 버전)
# ----------------------------
def col_has_any_value(s):
    """
    시리즈나 1-컬럼 DataFrame을 받아서 '하나라도 값이 있는지' 검사.
    - NaN/None -> 빈값
    - 공백 문자열 -> 빈값
    - 리스트/숫자/기타 객체도 str()로 변환 후 공백 판정
    """
    # 만약 DataFrame이 들어오면 모든 원소를 순회하도록 flatten
    if isinstance(s, pd.DataFrame):
        iterator = s.itertuples(index=False, name=None)
        for row in iterator:
            # row는 튜플일 수 있음: 각 원소 검사
            for val in row:
                if val is None:
                    continue
                v = str(val).strip()
                if v != "":
                    return True
        return False
    # Series인 경우 (일반적)
    for val in s:
        if val is None or (isinstance(val, float) and pd.isna(val)):
            continue
        if str(val).strip() != "":
            return True
    return False

# ----------------------------
# 예: 기존 rename_map 적용 (사용자 코드 그대로 유지)
# ----------------------------
rename_map = {
    "서비스ID": "서비스ID",
    "서비스명": "서비스명",
    "서비스 요약": "서비스개요",
    "서비스개요": "서비스개요",
    "상세 링크": "상세링크",
    "상세링크": "상세링크",
    "소관기관": "소관부처",
    "대표연락처": "대표연락처",
    "시작일": "시작일",
    "종료일": "종료일",
    "진행상태": "사업상태",
    "INTRS_THEMA_CD": "관심주제코드",
    "FMLY_CIRC_CD": "대상특성코드",
    "BKJR_LFTM_CYC_CD": "생애주기코드",
    "WLFARE_INFO_AGGRP_CD": "나이",
    "카테고리": "카테고리",
    "사업목적": "서비스목적",
    "지원대상": "지원대상상세",
    "지원대상상세": "지원대상상세",
    "지원내용": "서비스내용",
    "서비스내용": "서비스내용",
    "신청방법": "신청방법상세",
    "신청방법상세": "신청방법상세",
    "제출서류": "필요서류",
}

existing_map = {k:v for k,v in rename_map.items() if k in df.columns}
df = df.rename(columns=existing_map)

# ----------------------------
# 빈 칼럼 감지 및 삭제 (안전)
# ----------------------------
empty_cols = [c for c in df.columns if not col_has_any_value(df[c])]
if empty_cols:
    print("삭제할 빈 컬럼들:", empty_cols)
    df = df.drop(columns=empty_cols)
else:
    print("빈 컬럼 없음. 삭제할 항목 없음.")

# ----------------------------
# 저장
# ----------------------------
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print("저장 완료:", os.path.abspath(output_path))


민간복지 -최종

In [ ]:
import pandas as pd
import os
import re

# 입력 / 출력 파일 경로 (필요하면 경로를 수정)
INPUT_PATH = "private_final_welfare_list.csv"
OUTPUT_PATH = "private_final_welfare_list_with_category.csv"

# 1) 카테고리 키워드 사전 (체크리스트 기준)
category_keywords = {
    "생계지원": ["생계", "식비", "주거비", "공과금", "생활비", "긴급복지", "할인", "생활지원", "바우처"],
    "돌봄·보호": ["간병", "시설보호", "주야간", "돌봄", "보호", "양육", "방문간호", "돌봄서비스"],
    "건강·의료": ["질병", "진단", "치료", "의료", "병원", "건강검진", "의료비", "임신", "출산", "진료"],
    "일상생활": ["가사", "식사", "이동", "위생", "일상생활", "생활지원"],
    "안전위기": ["폭력", "학대", "자해", "위기", "긴급", "재난", "안전", "위험"],
    "주거지원": ["거처", "주거", "전세", "월세", "보증금", "주거환경", "주택", "임대", "주택수리"],
    "일자리": ["구직", "자활", "취업", "직업", "일자리", "직업능력", "훈련", "직업훈련"],
    "아동지원": ["아동", "보육", "양육", "유아", "영유아", "초등", "청소년"],
    "채무·법률": ["채무", "법률", "소송", "변호", "상담", "빚", "채권", "부채"],
    "고립·고독": ["고립", "고독", "외로움", "사회적고립", "고독사"]
}

# 2) 우선순위 (중복 매칭 시 이 순서로 먼저 선택)
priority_order = [
    "생계지원", "주거지원", "일자리", "아동지원",
    "돌봄·보호", "건강·의료", "안전위기", "채무·법률", "고립·고독"
]

# 3) 파일 읽기
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"입력 파일이 없습니다: {INPUT_PATH}")

df = pd.read_csv(INPUT_PATH, dtype=str)

# 4) 자동으로 검색할 텍스트 칼럼 후보 (존재하면 사용)
text_candidates = [
    "서비스개요", "서비스 요약", "서비스내용", "지원내용",
    "지원대상", "지원대상상세", "선정기준", "신청방법상세", "서비스명"
]
text_cols = [c for c in text_candidates if c in df.columns]

# 5) lower-case 키워드 준비
category_keywords_lower = {k: [kw.lower() for kw in kws] for k,kws in category_keywords.items()}

# 6) 행 텍스트 결합 함수
def combined_text_for_row(row):
    parts = []
    for c in text_cols:
        v = row.get(c, "")
        if pd.notna(v) and str(v).strip() != "":
            parts.append(str(v))
    return " ".join(parts).lower()  # 소문자로 변환하여 매칭 단순화

# 7) 카테고리 할당 함수
def assign_category(row):
    text = combined_text_for_row(row)
    matched = []
    for cat, kws in category_keywords_lower.items():
        for kw in kws:
            if kw in text:
                matched.append(cat)
                break
    # 우선순위에 따라 하나만 선택
    for cat in priority_order:
        if cat in matched:
            return cat
    # 아무것도 매칭되지 않으면 기본값 '생계지원'
    return "생계지원"

# 8) 카테고리 컬럼 생성 (덮어쓰기)
df["카테고리"] = df.apply(assign_category, axis=1)

# 9) 결과 간단 통계 출력
print("카테고리 분포:")
print(df["카테고리"].value_counts(dropna=False))

# 10) 저장
df.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print("저장 완료:", OUTPUT_PATH)
